In [1]:
# 環境設置
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys  # 鍵盤操作
from selenium.common.exceptions import NoSuchElementException, TimeoutException
# ---------- 等待頁面加載完成
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
# ----------
import pandas as pd
import pandas.io.formats.excel
import requests
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings()
from urllib.request import HTTPError
from bs4 import BeautifulSoup
import time

In [2]:
def organize_columns(df):
    # 處理 column 2：館藏地
    c2 = ["館藏地/室", "館藏室", "館藏地/館藏室", "館藏地", "典藏館", "館藏位置", "館藏地/區域", "典藏地名稱"]
    df["c2"] = ""
    for c in c2:
        try:
            df["c2"] += df[c]
        except:
            pass
    
    # 處理 column 3：索書號
    c3 = ["索書號", "索書號/期刊合訂本卷期", "索書號 / 部冊號"]
    df["c3"] = ""
    for c in c3:
        try:
            df["c3"] += df[c]
        except:
            pass

    # 處理 column 4：館藏狀態
    c4 = ["狀態/到期日", "目前狀態 / 到期日", "館藏狀態", "處理狀態", '狀態 (說明)', '館藏現況 說明', "目前狀態/預計歸還日期", "圖書狀況 / 到期日"]
    df["c4"] = ""
    for c in c4:
        try:
            df["c4"] += df[c]
        except:
            pass
    
    # 直接生成另一個 DataFrame
    test = pd.DataFrame()
    test["圖書館"] = df["圖書館"]
    test["館藏地"] = df["c2"]
    test["索書號"] = df["c3"]
    test["館藏狀態"] = df["c4"]
    test["連結"] = df["連結"]
    
    return test

In [3]:
my_options = Options()
my_options.add_argument('--incognito')  # 開啟無痕模式
my_options.add_argument('--start-maximized')  # 視窗最大化
# my_options.add_argument('--headless')  # 不開啟實體瀏覽器
capabilities = DesiredCapabilities.CHROME
capabilities['pageLoadStrategy'] = 'eager'

In [68]:
driver = webdriver.Chrome(options=my_options, desired_capabilities=capabilities)
def webpac_ajax_page_crawler(org, org_url, ISBN):
    try:
        # 進入＂搜尋主頁＂
        driver.get(org_url)
        # 等待點擊＂進階查詢＂按鈕，接著點擊
        WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.LINK_TEXT, '進階查詢'))).click()
        # 等待定位＂下拉式選單＂，選擇以 ISBN 方式搜尋
        search_field = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'as_type_1')))
        select = Select(search_field)
        select.select_by_value('i')
        # 定位＂搜尋欄＂，輸入 ISBN
        search_input = driver.find_element_by_id('as_keyword_1')
        search_input.send_keys(ISBN)
        search_input.send_keys(Keys.ENTER)  # 無法 submit()，用 send_keys(keys.ENTER) 來替代

        # 在＂搜尋結果頁面＂，等待定位＂詳細書目＂。
        # try-except 來判斷有沒有在＂搜尋結果頁面＂
        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.LINK_TEXT, '詳細書目')))
        except:
            try:
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.book-detail')))
                
                org_url = org_url.replace('/search.cfm', '')
                tgts = []
                url = driver.current_url
                mid = url.split('mid=')[-1].split('&')[0]
                ajax_page_url = f'{org_url}/ajax_page/get_content_area.cfm?mid={mid}&i_list_number=250&i_page=1&i_sory_by=1'
                tgt = pd.read_html(ajax_page_url, encoding='utf-8')[0]
                tgt['圖書館'], tgt['連結'] = org, url
                tgts.append(tgt)
                table = pd.concat(tgts, axis=0, ignore_index=True)
                table = organize_columns(table)
                return table  # 完成抓取 table
            except:  # 沒有搜尋結果，也沒有進入＂詳細書目頁面＂
                print(f'《{ISBN}》查無此書')
                return  # 什麼都不做，退出此 function
        
        # 抓取多個＂詳細書目＂的網址
        anchors = driver.find_elements_by_link_text('詳細書目')
        urls = []
        for anchor in anchors:
            urls.append(anchor.get_attribute('href'))

        # 抓取方式：找出 mid 後，進入 ajax pag 抓取 DataFrame
        org_url = org_url.replace('/search.cfm', '')
        tgts = []
        for url in urls:
            mid = url.split('mid=')[-1].split('&')[0]  # 抓取 mid
            ajax_page_url = f'{org_url}/ajax_page/get_content_area.cfm?mid={mid}&i_list_number=250&i_page=1&i_sory_by=1'
            tgt = pd.read_html(ajax_page_url, encoding='utf-8')[0]
            tgt["圖書館"], tgt["連結"] = org, url
            tgts.append(tgt)
        table = pd.concat(tgts, axis=0, ignore_index=True)
        table = organize_columns(table)
        return table  # 完成抓取 table
    except:
        print(f'《{ISBN}》無法爬取')